In [1]:
import pandas as pd
import numpy as np
import scipy as sc

In [2]:
sharesOutstanding = pd.read_csv("Shares Outstanding.csv")

In [3]:
sharesOutstanding = sharesOutstanding.iloc[:, 1:]

In [4]:
sharesOutstanding

,A,B,C,D,E,F,G,H,I
0,425000000,246970000,576250000,4230000000,1930000000,3370000000,16320000000,7510000000,508840000


In [5]:
data = pd.read_csv("Acutal Testing Data.csv")

In [6]:
data = data.iloc[:, 1:]

In [7]:
(Datarow, Datacol) = data.shape

In [8]:
#we use an extremely generic split first
Split = (70,30)

In [9]:
#splitting for train test set

train = data[0 : Datarow//10*7]
test = data[Datarow//10*7 : ]

In [10]:
(trainRow, trainCol) = train.shape
(testRow, testCol) = test.shape

In [11]:
prediction_1 = pd.read_csv("Predicted Testing Data Analyst 1.csv")
prediction_2 = pd.read_csv("Predicted Testing Data Analyst 2.csv")
prediction_3 = pd.read_csv("Predicted Testing Data Analyst 3.csv")

In [12]:
prediction_1 = prediction_1.iloc[:, 1:]
prediction_2 = prediction_2.iloc[:, 1:]
prediction_3 = prediction_3.iloc[:, 1:]

In [13]:
#prediction_1.head()

In [14]:
#prediction_2.head()

In [15]:
#prediction_3.head()

In [16]:
train.head()

,A,B,C,D,E,F,G,H,I
0,141.964897,226.352949,146.484337,77.539808,134.944262,32.982797,167.440304,297.091832,3091.003490
1,144.935575,227.882208,145.000811,78.897964,139.777286,33.608131,172.101489,305.086648,3101.989323
2,144.437134,234.666980,149.455145,81.378646,137.287929,33.521216,173.138616,304.388169,3134.173443
3,139.034096,231.480569,146.563009,83.680871,140.464937,32.373154,166.264334,300.779456,3122.525940
4,141.213983,233.275829,147.956318,79.816857,134.809211,31.813596,167.837537,312.957644,3134.811377


In [17]:
test.head()

,A,B,C,D,E,F,G,H,I
1750,913.101978,1607.232407,583.859981,106.878513,181.004480,154.274392,5995.375177,8308.623004,91405.65478
1751,924.252572,1645.848950,601.422483,109.950566,186.846766,160.151465,6002.865720,8242.598273,92811.13835
1752,905.048112,1709.280444,602.324912,106.962177,188.523359,160.884749,5981.571700,8359.614922,92887.80607
1753,912.758500,1695.297285,588.572990,103.279497,185.951549,152.867507,5975.784259,8204.472718,95897.24515
1754,932.547016,1677.149825,587.449461,102.705908,180.318411,157.815243,6153.334291,8301.332889,100716.85630


In [18]:
#note the parameters are np.arrays
#we do simplistic log return but need to change later
def getLogreturn(previousPrice, CurrentPrice):
    return np.log(CurrentPrice/previousPrice)

In [19]:
#we use the log difference of the return
stockreturn = np.log(train).diff()



In [20]:

stockreturn.head()

,A,B,C,D,E,F,G,H,I
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.020710,0.006733,-0.010179,0.017364,0.035189,0.018782,0.027457,0.026555,0.003548
2,-0.003445,0.029339,0.030257,0.030957,-0.017970,-0.002589,0.006008,-0.002292,0.010322
3,-0.038125,-0.013671,-0.019541,0.027898,0.022878,-0.034849,-0.040514,-0.011926,-0.003723
4,0.015557,0.007726,0.009462,-0.047276,-0.041097,-0.017436,0.009418,0.039691,0.003927


In [21]:
cov_matrix = stockreturn.cov()

In [22]:
cov_matrix

,A,B,C,D,E,F,G,H,I
A,0.000237,0.000138,0.000119,0.000099,0.000116,0.000105,0.000134,0.000120,0.000113
B,0.000138,0.000576,0.000179,0.000165,0.000167,0.000177,0.000236,0.000229,0.000191
C,0.000119,0.000179,0.000343,0.000139,0.000150,0.000148,0.000165,0.000149,0.000123
D,0.000099,0.000165,0.000139,0.000739,0.000513,0.000507,0.000137,0.000106,0.000104
E,0.000116,0.000167,0.000150,0.000513,0.000931,0.000555,0.000157,0.000130,0.000131
F,0.000105,0.000177,0.000148,0.000507,0.000555,0.000885,0.000166,0.000136,0.000124
G,0.000134,0.000236,0.000165,0.000137,0.000157,0.000166,0.000650,0.000272,0.000233
H,0.000120,0.000229,0.000149,0.000106,0.000130,0.000136,0.000272,0.000614,0.000219
I,0.000113,0.000191,0.000123,0.000104,0.000131,0.000124,0.000233,0.000219,0.000571


In [23]:
#shares outstanding and prices are 2 dataframes
def getMCapWeightsMatrix (SharesOutstanding, Prices): 
    totalMcap = SharesOutstanding.dot(Prices)
    
    #Mcap of each firm dataframe
    Mcap = SharesOutstanding.mul(Prices)
    #Mcap = Mcap.div(totalMcap)
    for stock in Mcap.columns:
        Mcap[stock] = Mcap[stock]/totalMcap
    
    return Mcap



In [24]:
currentPrice = train.iloc[10] #hardcode for testing. This should be a 9x1 vector

In [25]:
currentPrice.head()

A    141.775168
B    245.294243
C    155.017936
D     75.612571
E    135.101029
Name: 10, dtype: float64

In [26]:
riskadversionCoefficent = 3 #hardcoded for now

In [27]:
cov_matrix.shape

(9, 9)

In [28]:
Mcap1x9Matrix = getMCapWeightsMatrix(sharesOutstanding,currentPrice).values

In [29]:
Mcap1x9Matrix

array([[0.00732755, 0.00736718, 0.01086331, 0.0388959 , 0.03170921,
        0.01237441, 0.37441448, 0.3110686 , 0.20597937]])

In [30]:
MCapWeightVector = np.transpose(Mcap1x9Matrix)

In [31]:
MCapWeightVector

array([[0.00732755],
       [0.00736718],
       [0.01086331],
       [0.0388959 ],
       [0.03170921],
       [0.01237441],
       [0.37441448],
       [0.3110686 ],
       [0.20597937]])

In [32]:
ImpliedEquilibriumReturnVector = riskadversionCoefficent * cov_matrix.dot(MCapWeightVector)

In [33]:
ImpliedEquilibriumReturnVector

,0
A,0.000370
B,0.000660
C,0.000454
D,0.000482
E,0.000559
F,0.000546
G,0.001179
H,0.001056
I,0.000859


In [34]:
currentprice = currentPrice.values #convert to numpy array
currentprice

array([ 141.775168  ,  245.2942427 ,  155.0179365 ,   75.61257109,
        135.1010291 ,   30.19431058,  188.6527117 ,  340.6016952 ,
       3328.687648  ])

In [35]:
predicted_price1 = prediction_1.iloc[11] #one day after the previous day
predicted_price2 = prediction_2.iloc[11]
predicted_price3 = prediction_3.iloc[11]

In [36]:
predicted_price3

A     128.009745
B     321.469329
C     205.908514
D      95.204718
E      52.061439
F      39.076679
G      84.491354
H     328.813135
I    2759.588444
Name: 11, dtype: float64

In [37]:
#convert to numpy array
predictedArray1 = predicted_price1.to_numpy()
predictedArray2 = predicted_price2.to_numpy()
predictedArray3 = predicted_price3.to_numpy()


predictedReturn1 = getLogreturn(currentprice, predictedArray1)
predictedReturn2 = getLogreturn(currentprice, predictedArray2)
predictedReturn3 = getLogreturn(currentprice, predictedArray3)



In [38]:
P = np.array([predictedReturn1,predictedReturn2 ,predictedReturn3])

In [39]:
P.shape

(3, 9)

In [40]:
P

array([[ 0.06524868,  0.01352436, -0.52189693,  0.53877954, -0.58084905,
         0.03496488,  0.14319284, -0.33235942, -0.03760997],
       [-0.36735158,  0.04990293, -0.13152834,  0.09934027, -0.09373085,
         0.36017214, -0.26377242,  0.03044288,  0.15539809],
       [-0.10213609,  0.27044366,  0.28389114,  0.23040694, -0.95359832,
         0.25787233, -0.80325861, -0.03522414, -0.18749657]])